## Demo of LLM function calling.

- https://docs.litellm.ai/docs/completion/function_call

In [2]:
import json
import os
import random
import re
from textwrap import dedent
from typing import Dict

import litellm
import openai

# from docstring_parser import parse
from faker import Faker
from gait import a_message, function_to_tool, s_message, t_message, u_message
from pydantic import BaseModel, Field
from rich.pretty import pprint

In [3]:
litellm.drop_params = True
# litellm._turn_on_debug()  # 👈 this is the 1-line change you need to make

In [5]:
# print(litellm.supports_function_calling(model="ollama_chat/llama3.2:latest"))
# print(litellm.supports_function_calling(model="azure/gpt-4o-mini"))
# print(litellm.supports_function_calling(model="ollama_chat/qwen2:7b-instruct-q8_0"))
print(litellm.supports_function_calling(model="ollama_chat/phi4:14b-q8_0"))

False


In [6]:
fake = Faker()

In [7]:
def get_lat_lon(location: str) -> Dict:
    """Get the longitude and latitude of a given location.

    :param location: Can be a place, city, state, zipcode, state or country.
    """
    return {
        "location": self.location,
        "longitude": float(lon),
        "latitude": float(lat),
    }

In [8]:
# pprint(function_to_tool(get_lat_lon), expand_all=True)

In [9]:
class GetLatLon(BaseModel):
    """Get the latitude and longitude of a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()
        return {
            "location": self.location,
            "longitude": float(lon),
            "latitude": float(lat),
        }

In [10]:
pprint(GetLatLon(location=fake.city())(), expand_all=True)

{
│   'location': 'Wendymouth',
│   'longitude': 49.426997,
│   'latitude': -76.8629835
}

In [11]:
# pprint(openai.pydantic_function_tool(GetLatLon), expand_all=True)

In [12]:
class GetRoute(BaseModel):
    """Get the route between a starting latitude/longitude location and an ending latitude/longitude location."""

    lon1: float = Field(
        ...,
        description="The route starting longitude.",
    )
    lat1: float = Field(
        ...,
        description="The route starting latitude.",
    )
    lon2: float = Field(
        ...,
        description="The route ending longitude.",
    )
    lat2: float = Field(
        ...,
        description="The route ending latitude.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()

        if "scratchpad" in kwargs:
            kwargs["scratchpad"]["GetRoute"] = {"lon": lon, "lat": lat}

        return {
            "route": f"{self.lat1},{self.lon1} ---> {self.lat2},{self.lon2}",
        }

In [13]:
class GetCurrentTemperature(BaseModel):
    """Get the current temperature at a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )
    celsius_or_fahrenheit: str = Field(
        ...,
        description="The temperature in either 'C' for Celsius, or 'F' for Fahrenheit.",
    )

    def __call__(self, *args, **kwargs):
        temp = random.uniform(-5, 40)
        return {
            self.location: f"{temp:.1f}{self.celsius_or_fahrenheit}",
        }

In [14]:
tools = [
    openai.pydantic_function_tool(_)
    for _ in [
        GetCurrentTemperature,
        GetLatLon,
        GetRoute,
    ]
]

# pprint(tools, expand_all=True)

In [20]:
fake = Faker()

In [15]:
system = dedent(
    """
You are an AI expert in geo-spatial data analysis with access to specialized geo-spatial tools. Your task is to answer a user’s question, denoted as >>>question<<<, related to geo-spatial data. You will operate in a loop, alternating between reasoning about the problem and acting with tools as needed. At the end of the loop, you must output a clear, accurate, and well-supported answer.

Follow these guidelines to complete your task using the ReAct (Reasoning + Acting) pattern:
- **Reason**: Break down the >>>question<<< into logical steps. Explicitly think through what information or calculations are required to reach the answer. Document your reasoning before taking any action.
- **Act**: Use the appropriate geo-spatial tools to gather data, perform analysis, or compute results based on your reasoning. When calling tools:
    - ALWAYS provide the correct, specific arguments required by the tool (e.g., "40.7128, -74.0060" for coordinates, not "lat, lon").
    - Use explicit values rather than placeholders or variable names.
    - NEVER repeat a tool call with identical arguments if it was already executed; reuse the prior result instead.
- Alternate between reasoning and acting as needed to refine your approach and solve the problem systematically.
- If the >>>question<<< is unclear, reason through possible interpretations, make reasonable assumptions based on geo-spatial context, and state them in your response.
- Before finalizing your answer, review your reasoning and tool outputs to ensure accuracy and relevance to the >>>question<<<.

Begin now! For each iteration:
1. **Reason**: Explain your next step or hypothesis.
2. **Act**: Call the necessary tool(s) or process the data.
3. Repeat until the task is solved.

If you solve the task correctly, you will receive a virtual reward of $1,000,000.
"""
).strip()

In [70]:
# system = """
# You are an AI expert in geo-spatial data analysis with access to geo-spatial tools.
# You are tasked to answer a user >>>question<<<.
# You will run in a loop.
# At the end of the loop you output an answer.

# Here are the rules you should always follow to solve your task:
# - ALWAYS use the right arguments for the tools. NEVER use variable names.
# - ALWAYS use the values instead.
# - NEVER re-do a tool call that you previously did with the exact same arguments.

# Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.
# """.strip()

# prompt = f"""
# What's the current temperature in {fake.city()} in celsius and {fake.city()} in fahrenheit?
# And what is the route between them?
# """

# prompt = dedent(
#     f"""
# >>>What's the current temperature in {fake.city()} in celsius? and what are its coordinates?<<
# """
# ).strip()

prompt = dedent(
    f"""
>>>What's the route between {fake.city()} and {fake.city()}?<<
"""
).strip()


messages = [
    s_message(system),
    u_message(prompt),
]

In [89]:
for _ in messages:
    pprint(_, expand_all=True)

{
│   'role': 'system',
│   'content': 'You are an AI expert in geo-spatial data analysis with access to specialized geo-spatial tools. Your task is to answer a user’s question, denoted as >>>question<<<, related to geo-spatial data. You will operate in a loop, alternating between reasoning about the problem and acting with tools as needed. At the end of the loop, you must output a clear, accurate, and well-supported answer.\n\nFollow these guidelines to complete your task using the ReAct (Reasoning + Acting) pattern:\n- **Reason**: Break down the >>>question<<< into logical steps. Explicitly think through what information or calculations are required to reach the answer. Document your reasoning before taking any action.\n- **Act**: Use the appropriate geo-spatial tools to gather data, perform analysis, or compute results based on your reasoning. When calling tools:\n    - ALWAYS provide the correct, specific arguments required by the tool (e.g., "40.7128, -74.0060" for coordinates, not "lat, lon").\n    - Use explicit values rather than placeholders or variable names.\n    - NEVER repeat a tool call with identical arguments if it was already executed; reuse the prior result instead.\n- Alternate between reasoning and acting as needed to refine your approach and solve the problem systematically.\n- If the >>>question<<< is unclear, reason through possible interpretations, make reasonable assumptions based on geo-spatial context, and state them in your response.\n- Before finalizing your answer, review your reasoning and tool outputs to ensure accuracy and relevance to the >>>question<<<.\n\nBegin now! For each iteration:\n1. **Reason**: Explain your next step or hypothesis.\n2. **Act**: Call the necessary tool(s) or process the data.\n3. Repeat until the task is solved.\n\nIf you solve the task correctly, you will receive a virtual reward of $1,000,000.'
}

{
│   'role': 'user',
│   'content': ">>>What's the route between Gregoryland and Lake Deanshire?<<"
}

Message(
│   content='',
│   role='assistant',
│   tool_calls=[
│   │   ChatCompletionMessageToolCall(
│   │   │   function=Function(
│   │   │   │   arguments='{"location": "Gregoryland"}',
│   │   │   │   name='GetLatLon'
│   │   │   ),
│   │   │   id='4d373ba1-8c19-403d-9ca9-791470cdce4b',
│   │   │   type='function'
│   │   ),
│   │   ChatCompletionMessageToolCall(
│   │   │   function=Function(
│   │   │   │   arguments='{"location": "Lake Deanshire"}',
│   │   │   │   name='GetLatLon'
│   │   │   ),
│   │   │   id='1b70c4de-c4d7-4ca7-97f9-ba65cf87ca7d',
│   │   │   type='function'
│   │   )
│   ],
│   function_call=None,
│   provider_specific_fields=None
)

{
│   'role': 'tool',
│   'content': '{"location": "Gregoryland", "longitude": 149.7776, "latitude": 60.357177}',
│   'name': 'GetLatLon',
│   'tool_call_id': '4d373ba1-8c19-403d-9ca9-791470cdce4b'
}

{
│   'role': 'tool',
│   'content': '{"location": "Lake Deanshire", "longitude": 110.855344, "latitude": -6.8361165}',
│   'name': 'GetLatLon',
│   'tool_call_id': '1b70c4de-c4d7-4ca7-97f9-ba65cf87ca7d'
}

Message(
│   content='',
│   role='assistant',
│   tool_calls=[
│   │   ChatCompletionMessageToolCall(
│   │   │   function=Function(
│   │   │   │   arguments='{"lat1": 60.357177, "lat2": -6.8361165, "lon1": 149.7776, "lon2": 110.855344}',
│   │   │   │   name='GetRoute'
│   │   │   ),
│   │   │   id='f0a48cd6-8b95-4cb0-93c4-de5c7459a04e',
│   │   │   type='function'
│   │   )
│   ],
│   function_call=None,
│   provider_specific_fields=None
)

{
│   'role': 'tool',
│   'content': '{"route": "60.357177,149.7776 ---> -6.8361165,110.855344"}',
│   'name': 'GetLatLon',
│   'tool_call_id': 'f0a48cd6-8b95-4cb0-93c4-de5c7459a04e'
}

In [90]:
response = litellm.completion(
    model="ollama_chat/qwen2:7b-instruct-q8_0",
    # model="ollama_chat/llama3.2",
    # model="azure/gpt-4o-mini",
    # api_base=os.environ["AZURE_API_URL"] + "/gpt-4o-mini",
    messages=messages,
    tools=tools,
    # tool_choice="auto",  # auto is default, but we'll be explicit
    # parallel_tool_calls=False,
    temperature=0.0,
    top_p=1.0,
    n=1,
)

In [91]:
pprint(
    response.choices[0],
    expand_all=True,
)

Choices(
│   finish_reason='stop',
│   index=0,
│   message=Message(
│   │   content='The route between Gregoryland and Lake Deanshire is as follows: starting from the coordinates 60.357177,149.7776 in Gregoryland, you would travel to the coordinates -6.8361165,110.855344 in Lake Deanshire.',
│   │   role='assistant',
│   │   tool_calls=None,
│   │   function_call=None,
│   │   provider_specific_fields=None
│   )
)

In [92]:
messages.append(response.choices[0].message)

In [93]:
response.choices[0].message.content

'The route between Gregoryland and Lake Deanshire is as follows: starting from the coordinates 60.357177,149.7776 in Gregoryland, you would travel to the coordinates -6.8361165,110.855344 in Lake Deanshire.'

In [87]:
tool_call = response.choices[0].message.tool_calls[0]
pprint(tool_call, expand_all=True)

ChatCompletionMessageToolCall(
│   function=Function(
│   │   arguments='{"lat1": 60.357177, "lat2": -6.8361165, "lon1": 149.7776, "lon2": 110.855344}',
│   │   name='GetRoute'
│   ),
│   id='f0a48cd6-8b95-4cb0-93c4-de5c7459a04e',
│   type='function'
)

In [79]:
args = json.loads(tool_call.function.arguments)
result = GetLatLon(**args)()
messages.append(
    t_message(
        json.dumps(result),
        name="GetLatLon",
        tool_call_id=tool_call.id,
    )
)

In [88]:
args = json.loads(tool_call.function.arguments)
result = GetRoute(**args)()
messages.append(
    t_message(
        json.dumps(result),
        name="GetLatLon",
        tool_call_id=tool_call.id,
    )
)

In [ ]:
messages.append(
    t_message(
        json.dumps(GetCurrentTemperature(**args)()),
        name="GetCurrentTemperature",
        tool_call_id="32408a62-1520-4f0c-a38a-a5911e0e911c",
    )
)